In [1]:
import os
import sys

import pandas as pd
pd.options.display.max_columns = 999
import sqlalchemy as sqla
from sqlalchemy import create_engine

DB_URI = os.getenv('CD_DWH')
engine = create_engine(DB_URI)

In [2]:
QUERY = """
select
    distinct regexp_replace(lower(recipient_candidate_name), '[^a-z ]', '', 'g') as clean_name
from trg_analytics.candidate_contributions
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)

results.head(100)

,clean_name
0,None
1,west tony
2,krouse thomas
3,smith mel
4,denney ralph
5,perez v manuel
6,pacheco gayle e
7,smith robert w
8,leyva connie m
9,brough bill


In [4]:
from fuzzywuzzy import fuzz

size = len(results["clean_name"])

for i in range(size):
    name = results["clean_name"][i]
    for j in range(i + 1, size):
        test = results["clean_name"][j]
        fuz = fuzz.ratio(name, test)
        if fuz >= 85:
            print("{}% Similar {} to {} at {} and {}".format(fuz, name, test, i, j))

92% Similar leyva connie m to leyva connie at 8 and 1017
87% Similar eng michael to fong michael at 11 and 1307
93% Similar young steven r to young steven ray at 22 and 64
88% Similar young steven r to young steve at 22 and 170
85% Similar barton john h to burton john l at 23 and 719
92% Similar price curren d to price curren at 26 and 432
93% Similar peterson pete to peterson pete n at 27 and 268
93% Similar murray cynthia l to murray cynthia at 30 and 51
85% Similar torlakson mae to torlakson tom at 34 and 843
96% Similar papan virginia to papan virgina at 37 and 699
90% Similar wood carl w to wood carl at 40 and 1040
89% Similar song paul to fong paul at 54 and 154
86% Similar olberg r keith to olberg keith r at 58 and 892
95% Similar diaz manny to diaz mannny at 59 and 1179
94% Similar schaupp charles e to schaupp charles at 71 and 381
92% Similar marquez luis to marquez luis h at 81 and 332
86% Similar caserta dominic j to caserta jr dominic at 83 and 1299
92% Similar gardner m de

94% Similar shoemaker brandon k to shoemaker brandon at 1009 and 1543
87% Similar umberg tom to umberg thomas at 1011 and 1097
92% Similar vidal thomas h to vidal thomas at 1032 and 1123
91% Similar davenport ii robert to davenport robert at 1039 and 1174
93% Similar lutness carole l to lutness carole at 1052 and 1306
93% Similar umberg thomas j to umberg thomas at 1054 and 1097
90% Similar aldana jr manuel to aldana manuel at 1070 and 1526
93% Similar sanders joanne to sanders joanne b at 1102 and 1356
90% Similar munn john to munn john r at 1111 and 1199
93% Similar groper richard p to groper richard at 1113 and 1114
92% Similar atkins toni g to atkins toni at 1119 and 1389
85% Similar camejo peter m to camejo peter miguel at 1150 and 1324
96% Similar mccann john to mc cann john at 1170 and 1240
94% Similar jacobi veronica to jacobi veronica s at 1202 and 1205
92% Similar houston guy to houston guy s at 1230 and 1491
93% Similar walker prophet to walker prophet l at 1249 and 1519
91%